In [3]:
"""
Teams Calendar

Use schedule_and_results table in GCP to extract a complete calendar for each team where they stand in their points & stats

Save results in Google Big Query
"""

credential_keys = '/Users/antoinetl/Documents/nhl_prediction/nhl_prediction/account_keys/hockey-prediction-qc-9c75aa8a78f9.json'

In [4]:
# Import the important packages
import pandas as pd
import numpy as np
import datetime as dt

# Connexion GBQ
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(credential_keys,)
pandas_gbq.context.credentials = credentials
# https://pandas-gbq.readthedocs.io/en/latest/intro.html

In [5]:
# Set project_id to your Google Cloud Platform project ID.
project_id = "hockey-prediction-qc"


sql = """
SELECT *
from `hockey_prediction_qc.historical_games_detailed`
where gameType = 'R' 
    and status_abstractGameState = 'Final'
order by date;
"""

df_raw = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████| 1271/1271 [00:00<00:00, 1419.62rows/s]


In [6]:
df = df_raw.sort_values(by=['season', 'date'])

#cols = ['team', 'season', 'date', 'games_played']
d = [] 

# get seasons in dataset - Loop over seasons
seasons = df['season'].explode().unique()
for season in seasons:
    
    df_season = df[df['season'] == season]
    
    # get teams who played in season - Loop over teams
    teams = df_season['teams_home_team_id'].explode().unique()
    for team in teams:
                
        # only select team's game
        df_team = df_season[(df_season['teams_home_team_id'] == team )| (df_season['teams_away_team_id'] == team)]
        df_team['date'] = df_team['date'].astype('datetime64[ns]')

        # Begining and end for each season
        start_date = df_team['date'].min()
        end_date = df_team['date'].max()
        
        delta = dt.timedelta(days=1)
        
        # Loop over days in season
        while start_date <= end_date:
            
            df_team.sort_values('date');
            
            df_team_day = df_team[df_team['date'] <= start_date]
            
            
            
            # stats home 
            
            
            df_team_day_home = df_team_day[df_team_day['teams_home_team_id'] == team]
            
            # games played
            index = df_team_day_home.index
            games_played_home = len(index)
            
            home_shots_for = df_team_day_home['home_teamStats_teamSkaterStats_shots'].sum()
            home_shots_against = df_team_day_home['away_teamStats_teamSkaterStats_shots'].sum()
            home_shots_blocked = df_team_day_home['home_teamStats_teamSkaterStats_blocked'].sum()
            home_goals_for = df_team_day_home['home_teamStats_teamSkaterStats_goals'].sum()
            home_goals_against = df_team_day_home['away_teamStats_teamSkaterStats_goals'].sum()
            home_hits = df_team_day_home['home_teamStats_teamSkaterStats_hits'].sum()
            home_pim = df_team_day_home['home_teamStats_teamSkaterStats_pim'].sum() # penality minutes
            home_powerPlayGoals_for = df_team_day_home['home_teamStats_teamSkaterStats_powerPlayGoals'].sum() #power play goals
            home_powerPlayGoals_against = df_team_day_home['away_teamStats_teamSkaterStats_powerPlayGoals'].sum()
            home_powerPlayOpportunities = df_team_day_home['home_teamStats_teamSkaterStats_powerPlayOpportunities'].sum()
            home_takeaways = df_team_day_home['home_teamStats_teamSkaterStats_takeaways'].sum()
            home_giveaways = df_team_day_home['home_teamStats_teamSkaterStats_giveaways'].sum()

            
            
            # stats away
            
            
            df_team_day_away = df_team_day[df_team_day['teams_away_team_id'] == team]
            
            # games played
            index = df_team_day_away.index
            games_played_away = len(index)
            
            away_shots_for = df_team_day_away['away_teamStats_teamSkaterStats_shots'].sum()
            away_shots_against = df_team_day_away['home_teamStats_teamSkaterStats_shots'].sum()
            away_shots_blocked = df_team_day_away['away_teamStats_teamSkaterStats_blocked'].sum()
            away_goals_for = df_team_day_away['away_teamStats_teamSkaterStats_goals'].sum()
            away_goals_against = df_team_day_away['away_teamStats_teamSkaterStats_goals'].sum()
            away_hits = df_team_day_away['away_teamStats_teamSkaterStats_hits'].sum()
            away_pim = df_team_day_away['away_teamStats_teamSkaterStats_pim'].sum() # penality minutes
            away_powerPlayGoals_for = df_team_day_away['away_teamStats_teamSkaterStats_powerPlayGoals'].sum() #power play goals
            away_powerPlayGoals_against = df_team_day_away['home_teamStats_teamSkaterStats_powerPlayGoals'].sum()
            away_powerPlayOpportunities = df_team_day_away['away_teamStats_teamSkaterStats_powerPlayOpportunities'].sum()
            away_takeaways = df_team_day_away['away_teamStats_teamSkaterStats_takeaways'].sum()
            away_giveaways = df_team_day_away['away_teamStats_teamSkaterStats_giveaways'].sum()
            
            
            
            # total
            
            games_played = games_played_home + games_played_away
            shots_for = home_shots_for + away_shots_for
            shots_against = home_shots_against + away_shots_against
            shots_blocked = home_shots_blocked + away_shots_blocked
            goals_for = home_goals_for + away_goals_for
            goals_against = home_goals_against + away_goals_against
            hits = home_hits + away_hits
            pim = home_pim + away_pim
            powerPlayGoals_for = home_powerPlayGoals_for + away_powerPlayGoals_for
            powerPlayGoals_against = home_powerPlayGoals_against + away_powerPlayGoals_against
            powerPlayOpportunities = home_powerPlayOpportunities + away_powerPlayOpportunities
            takeaways = home_takeaways + away_takeaways
            giveaways = home_giveaways + away_giveaways
            
            
            # total / AVERAGES
            shots_for_avg = (home_shots_for + away_shots_for) / games_played
            shots_against_avg = (home_shots_against + away_shots_against) / games_played
            shots_blocked_avg = (home_shots_blocked + away_shots_blocked) / games_played
            goals_for_avg = (home_goals_for + away_goals_for) / games_played
            goals_against_avg = (home_goals_against + away_goals_against) / games_played
            hits_avg = (home_hits + away_hits) / games_played
            pim_avg = (home_pim + away_pim) / games_played
            powerPlayGoals_for_avg = (home_powerPlayGoals_for + away_powerPlayGoals_for) / games_played
            powerPlayGoals_against_avg = (home_powerPlayGoals_against + away_powerPlayGoals_against) / games_played
            powerPlayOpportunities_avg = (home_powerPlayOpportunities + away_powerPlayOpportunities) / games_played
            takeaways_avg = (home_takeaways + away_takeaways) / games_played
            giveaways_avg = (home_giveaways + away_giveaways) / games_played
            
            
            # team_stats
            
            
            # get last's night game id
            df_stats_max_1 = pd.concat([df_team_day.tail(2)])
            last_game_id = df_stats_max_1['gameID'].values[0]
            
            
            #create df with only last game by team to date
            df_stats_max = pd.concat([df_team_day.tail(1)])
            tonights_game_id = df_stats_max['gameID'].values[0]
            
            if df_stats_max['teams_home_team_id'].values[0] == team :
                n_wins = df_stats_max['teams_home_leagueRecord_wins'].values[0]
                n_losses = df_stats_max['teams_home_leagueRecord_losses'].values[0]
                n_ot = df_stats_max['teams_home_leagueRecord_ot'].values[0]
            else :
                n_wins = df_stats_max['teams_away_leagueRecord_wins'].values[0]
                n_losses = df_stats_max['teams_away_leagueRecord_losses'].values[0]
                n_ot = df_stats_max['teams_away_leagueRecord_ot'].values[0]
            
            points = (3 * n_wins) + (1 * n_ot)
            ppg = points / games_played
            

            
            d.append(
                {
                    'team': team,
                    'season': season,
                    'date':  start_date,
                    'games_played': games_played,
                    'tonights_game_id': tonights_game_id,
                    'last_game_id': last_game_id,
                    'wins': n_wins,
                    'losses': n_losses,
                    'ot': n_ot,
                    'points' : points,
                    'ppg' : ppg,
                    'shots_for_avg' : shots_for_avg,
                    'shots_against_avg' : shots_against_avg,
                    'shots_blocked_avg' : shots_blocked_avg,
                    'goals_for_avg' : goals_for_avg,
                    'goals_against_avg' : goals_against_avg,
                    'hits_avg' : hits_avg,
                    'pim_avg' : pim_avg,
                    'powerPlayGoals_for_avg' : powerPlayGoals_for_avg,
                    'powerPlayGoals_against_avg' : powerPlayGoals_against_avg,
                    'powerPlayOpportunities_avg' : powerPlayOpportunities_avg,
                    'takeaways_avg' : takeaways_avg,
                    'giveaways_avg' : giveaways_avg
                }
            )
            
            start_date += delta
        
        print("it's done for team " , team, " for season ", season)

    

calendar_stats = pd.DataFrame(d)

<ipython-input-6-8b7cf7d87569>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_team['date'] = df_team['date'].astype('datetime64[ns]')


it's done for team  22  for season  20172018
it's done for team  52  for season  20172018
it's done for team  5  for season  20172018
it's done for team  28  for season  20172018
it's done for team  6  for season  20172018
it's done for team  24  for season  20172018
it's done for team  16  for season  20172018
it's done for team  7  for season  20172018
it's done for team  26  for season  20172018
it's done for team  9  for season  20172018
it's done for team  17  for season  20172018
it's done for team  3  for season  20172018
it's done for team  14  for season  20172018
it's done for team  29  for season  20172018
it's done for team  25  for season  20172018
it's done for team  12  for season  20172018
it's done for team  13  for season  20172018
it's done for team  23  for season  20172018
it's done for team  2  for season  20172018
it's done for team  53  for season  20172018
it's done for team  19  for season  20172018
it's done for team  10  for season  20172018
it's done for te

In [7]:
calendar_stats.loc[calendar_stats['games_played'] == 1, 'last_game_id'] = ""

pd.set_option('display.max_columns', None)
calendar_stats.head()

,team,season,date,games_played,tonights_game_id,last_game_id,wins,losses,ot,points,ppg,shots_for_avg,shots_against_avg,shots_blocked_avg,goals_for_avg,goals_against_avg,hits_avg,pim_avg,powerPlayGoals_for_avg,powerPlayGoals_against_avg,powerPlayOpportunities_avg,takeaways_avg,giveaways_avg
0,22,20172018,2017-10-04,1,2017020003,,1,0,0.0,3.0,3.0,45.0,27.0,18.0,3.0,0.0,31.0,7.0,0.0,0.0,2.0,18.0,16.0
1,22,20172018,2017-10-05,1,2017020003,,1,0,0.0,3.0,3.0,45.0,27.0,18.0,3.0,0.0,31.0,7.0,0.0,0.0,2.0,18.0,16.0
2,22,20172018,2017-10-06,1,2017020003,,1,0,0.0,3.0,3.0,45.0,27.0,18.0,3.0,0.0,31.0,7.0,0.0,0.0,2.0,18.0,16.0
3,22,20172018,2017-10-07,2,2017020028,2017020003,1,1,0.0,3.0,1.5,40.0,26.5,12.5,2.5,1.0,28.5,13.0,0.5,0.5,3.5,11.0,9.0
4,22,20172018,2017-10-08,2,2017020028,2017020003,1,1,0.0,3.0,1.5,40.0,26.5,12.5,2.5,1.0,28.5,13.0,0.5,0.5,3.5,11.0,9.0


In [8]:
def create_lag_wlotp(df, p):
    """
    Cette fonction prend comme intrant un df, une liste de variables pour lesquelles il faut
    creer des valeurs retardees, le nombre de retard, les variables sur lesquelles il faut sort,
    les variables de partition et la liste de varibles pour lesquelles on veut des valeurs retardees.
    La fonction retourne un df augmente des variables retardees
    """
    df['wins_last_{}'.format(p)] = 0
    df['loss_last_{}'.format(p)] = 0
    df['ot_last_{}'.format(p)] = 0
    df['points_last_{}'.format(p)] = 0
    df['ppg_last_{}'.format(p)] = 0

    for index, row in df.iterrows():
        gameId = row['games_played'] - p
        team = row['team']
        season = row['season']

        wins_now = row['wins']
        loss_now = row['losses']
        ot_now = row['ot']
        points_now = row['points']
        
        
        if gameId >= 1:
            df2 =  df[(df['games_played'] == gameId) & (df['team'] == team) & (df['season'] == season)]
            
            wins_then = df2['wins'].mean()
            loss_then = df2['losses'].mean()
            ot_then = df2['ot'].mean()
            points_then = df2['points'].mean()
        
        else:
            wins_then = 0
            loss_then = 0
            ot_then = 0
            points_then = 0
    
        wins = wins_now - wins_then
        loss = loss_now - loss_then
        ot = ot_now - ot_then
        points = points_now - points_then
        ppg = points / p
        
        df.loc[index, 'wins_last_{}'.format(p)] = wins
        df.loc[index, 'loss_last_{}'.format(p)] = loss
        df.loc[index, 'ot_last_{}'.format(p)] = ot
        df.loc[index, 'points_last_{}'.format(p)] = points
        df.loc[index, 'ppg_last_{}'.format(p)] = ppg
    
    return df

In [9]:
calendar_stats = create_lag_wlotp(calendar_stats, 1)
calendar_stats = create_lag_wlotp(calendar_stats, 10)

In [10]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "hockey-prediction-qc"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'hockey_prediction_qc.teams_calendar'

pandas_gbq.to_gbq(calendar_stats, table_id, project_id=project_id, if_exists='replace')

1it [00:06,  6.15s/it]
